# History

### Ver2
* VIF 분석을 통해 피처 선택을 수행하고 테스트합니다.
* VIF 분석 결과 제거할 피처가 없어 permutation_importance 함수를 사용 후 제거할 피쳐를 선택합니다.
* Perform feature selection through VIF analysis and test it.
* After conducting the VIF analysis and finding no features to remove, I will now use the permutation_importance function to determine which features to remove.

### Ver3
* RNF와 Type컬럼을 제거한 데이터에 대해서 모델을 학습하고 튜닝합니다.
* Train and fine-tune the model using the data excluding the 'RNF' and 'Type' columns.

In [2]:
# !pip install git+https://github.com/tooha289/DataAnalysisLibrary.git

  Cloning https://github.com/tooha289/DataAnalysisLibrary.git to c:\users\medici\appdata\local\temp\pip-req-build-7hoxqnkx
  Resolved https://github.com/tooha289/DataAnalysisLibrary.git to commit e65d47d5b1b85d6d4f87376519b594cd106a596d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for DataAnalysisLibrary: filename=DataAnalysisLibrary-0.2.4-py3-none-any.whl size=7527 sha256=acfb3649b7b22aa5a214d6454630b687fb207244d503a6c9f43031297f440f04
  Stored in directory: C:\Users\medici\AppData\Local\Temp\pip-ephem-wheel-cache-he046s58\wheels\8e\c1\d0\93595a944e501ada8c98086faf85afafe77739b46011f1a002
Successfully built DataAnalysisLibrary


  Running command git clone --filter=blob:none --quiet https://github.com/tooha289/DataAnalysisLibrary.git 'C:\Users\medici\AppData\Local\Temp\pip-req-build-7hoxqnkx'


# 사용 모듈 Import

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from DataAnalysis import eda
from DataAnalysis import feature_engineering

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.optimizers import Adam

import optuna

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score

# 0. Check GPU

In [5]:
import tensorflow as tf

# GPU 디바이스 목록 가져오기
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# GPU 사용 가능한 경우
if gpu_devices:
    for device in gpu_devices:
        print("GPU 사용 가능:", device)
else:
    print("GPU 사용 불가능")

GPU 사용 가능: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# 1. Load & Check Data

In [6]:
df = pd.read_csv('../data/train.csv')
df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,136424,M22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,136425,H38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,136426,L54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0
136427,136427,L53876,L,301.7,310.9,1447,46.3,42,0,0,0,0,0,0


In [7]:
test_df = pd.read_csv('../data/test.csv')
test_df

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,0,0,0,0,0
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,0,0,0,0,0
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,0,0,0,0,0
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,0,0,0,0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       136429 non-null  int64  
 1   Product ID               136429 non-null  object 
 2   Type                     136429 non-null  object 
 3   Air temperature [K]      136429 non-null  float64
 4   Process temperature [K]  136429 non-null  float64
 5   Rotational speed [rpm]   136429 non-null  int64  
 6   Torque [Nm]              136429 non-null  float64
 7   Tool wear [min]          136429 non-null  int64  
 8   Machine failure          136429 non-null  int64  
 9   TWF                      136429 non-null  int64  
 10  HDF                      136429 non-null  int64  
 11  PWF                      136429 non-null  int64  
 12  OSF                      136429 non-null  int64  
 13  RNF                      136429 non-null  int64  
dtypes: f

In [9]:
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,136429.0,NaN,NaN,NaN,68214.0,39383.804275,0.0,34107.0,68214.0,102321.0,136428.0
Product ID,136429,9976,L53257,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,136429,3,L,95354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Air temperature [K],136429.0,NaN,NaN,NaN,299.862776,1.862247,295.3,298.3,300.0,301.2,304.4
Process temperature [K],136429.0,NaN,NaN,NaN,309.94107,1.385173,305.8,308.7,310.0,310.9,313.8
Rotational speed [rpm],136429.0,NaN,NaN,NaN,1520.33111,138.736632,1181.0,1432.0,1493.0,1580.0,2886.0
Torque [Nm],136429.0,NaN,NaN,NaN,40.348643,8.502229,3.8,34.6,40.4,46.1,76.6
Tool wear [min],136429.0,NaN,NaN,NaN,104.408901,63.96504,0.0,48.0,106.0,159.0,253.0
Machine failure,136429.0,NaN,NaN,NaN,0.015744,0.124486,0.0,0.0,0.0,0.0,1.0
TWF,136429.0,NaN,NaN,NaN,0.001554,0.039389,0.0,0.0,0.0,0.0,1.0


In [10]:
df.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

# 2. Separate columns

In [11]:
feature_cols = df.columns.difference(['id', 'Product ID', 'Machine failure'])
numeric_cols = df.columns.difference(['id', 'Product ID', 'Machine failure', 'Type'])
continuos_cols = feature_cols.difference(["TWF", "HDF", "PWF", "OSF", "RNF", "Type"])
discrete_cols = feature_cols.difference(continuos_cols)

# 3. FeatureEngineering

## 3.1 Preprocessing

In [12]:
dfp = feature_engineering.DataFramePreprocessor()
le = LabelEncoder()
stds = StandardScaler()

### 3.1.1 Train set

In [13]:
x_data = df.copy()
x_data = x_data.drop(labels=['id', 'Product ID'], axis=1)
x_data, _ = dfp.fit_transform_multiple_transformer(x_data, [le, stds], [["Type"], continuos_cols])
x_data = pd.concat([x_data, df[discrete_cols.difference(['Type'])]], axis=1)

x_data[discrete_cols] = x_data[discrete_cols].astype("int8")
x_data

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,0.395880,-0.246230,0.545416,0.556417,-0.499711,0,0,0,0,0
1,2,1.469856,1.558605,1.720308,1.494433,-1.323028,0,0,0,0,0
2,1,-0.302204,-1.040358,2.051873,-1.241447,-1.628831,0,0,0,0,0
3,1,0.610675,0.692284,0.026445,1.447532,0.464745,0,0,0,0,0
4,2,-1.000288,-0.679391,0.869773,-1.100744,-0.582043,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
136424,2,0.127386,1.053251,0.069693,1.650769,-0.335048,0,0,0,0,0
136425,0,-1.268781,-1.040358,-0.528565,-1.601020,1.029305,0,0,0,0,0
136426,1,0.342182,1.342025,0.026445,1.713303,-0.217431,0,0,0,0,0
136427,1,0.986567,0.692284,-0.528565,-0.975676,0.699979,0,0,0,0,0


In [14]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Type                     136429 non-null  int8   
 1   Air temperature [K]      136429 non-null  float64
 2   Process temperature [K]  136429 non-null  float64
 3   Rotational speed [rpm]   136429 non-null  float64
 4   Tool wear [min]          136429 non-null  float64
 5   Torque [Nm]              136429 non-null  float64
 6   HDF                      136429 non-null  int8   
 7   OSF                      136429 non-null  int8   
 8   PWF                      136429 non-null  int8   
 9   RNF                      136429 non-null  int8   
 10  TWF                      136429 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 6.0 MB


In [15]:
y_data = df.copy().pop('Machine failure')
y_data = y_data.astype("int8")
y_data

0         0
1         0
2         0
3         0
4         0
         ..
136424    0
136425    0
136426    0
136427    0
136428    0
Name: Machine failure, Length: 136429, dtype: int8

### 3.1.2 Test set

In [16]:
x_test = test_df.copy()
x_test = x_test.drop(labels=['id', 'Product ID'], axis=1)
x_test, _ = dfp.fit_transform_multiple_transformer(x_test, [le, stds], [["Type"], continuos_cols])
x_test = pd.concat([x_test, test_df[discrete_cols.difference(['Type'])]], axis=1)

x_test[discrete_cols] = x_test[discrete_cols].astype("int8")
x_test

,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Tool wear [min],Torque [Nm],HDF,OSF,PWF,RNF,TWF
0,1,1.313830,1.126570,-0.153806,-0.693494,-0.274579,0,0,0,0,0
1,1,0.990824,0.765635,1.375097,-1.366729,-1.356341,0,0,0,0,0
2,1,0.775487,0.332512,0.031949,-0.129855,-0.309854,0,0,0,0,0
3,2,0.129475,-0.244985,-0.296694,-1.554608,0.854217,0,0,0,0,0
4,2,1.906008,1.704068,-0.039496,0.151964,0.113445,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
90949,1,1.313830,1.054383,-0.260972,-1.398042,0.007620,0,0,0,0,0
90950,1,-1.054879,-0.100611,0.153403,-1.147536,-0.768427,0,0,0,0,0
90951,1,-2.293068,-2.699348,-0.139517,1.294897,0.125203,0,0,0,0,0
90952,1,-0.947211,-1.544354,0.096248,-0.552584,-0.004138,0,0,0,0,0


In [17]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Type                     90954 non-null  int8   
 1   Air temperature [K]      90954 non-null  float64
 2   Process temperature [K]  90954 non-null  float64
 3   Rotational speed [rpm]   90954 non-null  float64
 4   Tool wear [min]          90954 non-null  float64
 5   Torque [Nm]              90954 non-null  float64
 6   HDF                      90954 non-null  int8   
 7   OSF                      90954 non-null  int8   
 8   PWF                      90954 non-null  int8   
 9   RNF                      90954 non-null  int8   
 10  TWF                      90954 non-null  int8   
dtypes: float64(5), int8(6)
memory usage: 4.0 MB


## 3.2 Feature selecting

### 3.2.1 VIF analysis

In [18]:
fs = feature_engineering.FeatureSelector()

In [19]:
formula_cols = pd.Series(feature_cols).apply(lambda col: f"Q('{col}')")
formula_cols

0         Q('Air temperature [K]')
1                         Q('HDF')
2                         Q('OSF')
3                         Q('PWF')
4     Q('Process temperature [K]')
5                         Q('RNF')
6      Q('Rotational speed [rpm]')
7                         Q('TWF')
8             Q('Tool wear [min]')
9                 Q('Torque [Nm]')
10                       Q('Type')
dtype: object

In [20]:
formula = "Q('Machine failure')~" + "+".join(formula_cols)
formula+="-1"
formula

"Q('Machine failure')~Q('Air temperature [K]')+Q('HDF')+Q('OSF')+Q('PWF')+Q('Process temperature [K]')+Q('RNF')+Q('Rotational speed [rpm]')+Q('TWF')+Q('Tool wear [min]')+Q('Torque [Nm]')+Q('Type')-1"

In [21]:
data = pd.concat([x_data, y_data], axis=1)

In [22]:
fs.get_vif_dataframe(formula, data)

,features,VIF Factor
0,Q('Air temperature [K]'),3.806145
4,Q('Process temperature [K]'),3.772470
9,Q('Torque [Nm]'),2.629339
6,Q('Rotational speed [rpm]'),2.608535
1,Q('HDF'),1.038264
3,Q('PWF'),1.037175
2,Q('OSF'),1.030903
10,Q('Type'),1.011599
8,Q('Tool wear [min]'),1.006448
7,Q('TWF'),1.006107


### 3.2.2 Get permutation importance

In [23]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [24]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train.values, y_train.values)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [25]:
fs.get_permutation_importance(dtc, x_valid, y_valid)

,feature,perm_importance
0,Type,0.000134
1,Air temperature [K],0.008563
2,Process temperature [K],0.008122
3,Rotational speed [rpm],0.002798
4,Tool wear [min],0.000636
5,Torque [Nm],0.003258
6,HDF,0.008200
7,OSF,0.006367
8,PWF,0.003914
9,RNF,0.000019


* 위 결과를 바탕으로 우선 RNF값을 제거하고 성능을 평가합니다.
* 그 후 Type에 대해서도 제거를 수행해봅니다.

* 모든 데이터 사용시

In [26]:
# Accuracy: 0.9960052774316499, f1_score: 0.8593548387096774, Roc-Auc: 0.8878698655479385
# Accuracy: 0.9960052774316499, f1_score: 0.858257477243173, Roc-Auc: 0.8835533699108423
# Accuracy: 0.9965916587260867, f1_score: 0.8824273072060682, Roc-Auc: 0.9055905397260844
# Accuracy: 0.9957120867844316, f1_score: 0.845442536327609, Roc-Auc: 0.8719626985611462
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85956275, 0.88643314])

In [27]:
x_data.drop('RNF', axis=1, inplace=True)
x_test.drop('RNF', axis=1, inplace=True)

* RNF 제거 시 스코어

In [28]:
# Accuracy: 0.9960052774316499, f1_score: 0.8597168597168596, Roc-Auc: 0.8890167495931356
# Accuracy: 0.9960419262625523, f1_score: 0.859375, Roc-Auc: 0.8835719877243662
# Accuracy: 0.9966649563878912, f1_score: 0.883780332056194, Roc-Auc: 0.9021952567006809
# Accuracy: 0.9956021402917247, f1_score: 0.841688654353562, Roc-Auc: 0.8707626722364238
# Accuracy: 0.9958218801539307, f1_score: 0.8523316062176166, Roc-Auc: 0.8831892340605477
# array([0.99602724, 0.85937849, 0.88574718])

In [29]:
x_data.drop('Type', axis=1, inplace=True)
x_test.drop('Type', axis=1, inplace=True)

* 추가로 Type 제거 시 스코어

In [30]:
# Accuracy: 0.9960419262625523, f1_score: 0.8608247422680411, Roc-Auc: 0.8890353667134395
# Accuracy: 0.9960785750934545, f1_score: 0.8601307189542484, Roc-Auc: 0.8824464326537399
# Accuracy: 0.9966649563878912, f1_score: 0.8834827144686299, Roc-Auc: 0.9010510838165307
# Accuracy: 0.9955288426299201, f1_score: 0.8390501319261213, Roc-Auc: 0.8695812637252253
# Accuracy: 0.995968480850284, f1_score: 0.8567708333333334, Roc-Auc: 0.8832637053146436
# array([0.99605656, 0.86005183, 0.88507557])

# 4. Modeling

## 4.1 LogisticRegression

In [31]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [32]:
lr = LogisticRegression()
lr.fit(x_train.values, y_train.values)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
acc_score = lr.score(x_train.values, y_train)
acc_score

0.9961200535569433

In [34]:
f1score = f1_score(y_train, lr.predict(x_train.values))
f1score

0.8606528606528607

In [35]:
acc_score = lr.score(x_valid.values, y_valid)
acc_score

0.9961592588249091

In [36]:
f1score = f1_score(y_valid, lr.predict(x_valid.values))
f1score

0.8613756613756614

## 4.2 XGBoost

In [36]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'tree_method': 'gpu_hist',  # GPU 가속 사용
        'gpu_id': 0,  # 사용할 GPU 장치 ID 설정 (0 또는 다른 GPU ID)
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    valid_score = []

    for train_index, test_index in skf.split(x_data, y_data):
        x_train, x_valid = x_data.iloc[train_index], x_data.iloc[test_index]
        y_train, y_valid = y_data.iloc[train_index], y_data.iloc[test_index]

        xgb = XGBClassifier(**params)
        # Early stopping 설정
        xgb.fit(x_train.values, y_train.values, verbose=0, eval_set=[(x_valid.values, y_valid.values)],
                early_stopping_rounds=10)

        acc_score = xgb.score(x_valid, y_valid)
        predict_val = xgb.predict(x_valid)
        f1score = f1_score(y_valid, predict_val)
        roc_auc = roc_auc_score(y_valid, predict_val)
        valid_score.append((acc_score, f1score, roc_auc))

    # Calculate and report the mean F1-score for this trial
    mean_f1_score = sum([score[1] for score in valid_score]) / len(valid_score)
    trial.report(mean_f1_score, step=trial.number)

    return mean_f1_score

In [37]:
# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')  # Maximize the F1-score
study.optimize(objective, n_trials=50)  # Number of trials can be adjusted

# Get the best parameters
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# Get the best F1-score achieved during the optimization
best_f1_score = study.best_value
print(f"Best F1-Score: {best_f1_score}")

[I 2023-09-22 15:05:49,308] A new study created in memory with name: no-name-0e8c3106-c795-4755-bbc4-2634c8ba16c7
[I 2023-09-22 15:06:01,414] Trial 0 finished with value: 0.8606431963526007 and parameters: {'n_estimators': 492, 'max_depth': 17, 'learning_rate': 0.06632124274739234, 'subsample': 0.5726541539816163, 'colsample_bytree': 0.7363022189820154}. Best is trial 0 with value: 0.8606431963526007.
[I 2023-09-22 15:06:26,536] Trial 1 finished with value: 0.8624585670942526 and parameters: {'n_estimators': 761, 'max_depth': 8, 'learning_rate': 0.01710992525291977, 'subsample': 0.8545136758613967, 'colsample_bytree': 0.8646143849192524}. Best is trial 1 with value: 0.8624585670942526.
[I 2023-09-22 15:06:45,232] Trial 2 finished with value: 0.8600123549731066 and parameters: {'n_estimators': 399, 'max_depth': 20, 'learning_rate': 0.0361502483262547, 'subsample': 0.8922688607022589, 'colsample_bytree': 0.913892955135641}. Best is trial 1 with value: 0.8624585670942526.
[I 2023-09-22 15

[I 2023-09-22 15:21:40,580] Trial 28 finished with value: 0.862770544775411 and parameters: {'n_estimators': 900, 'max_depth': 3, 'learning_rate': 0.01484599338033089, 'subsample': 0.7459229095123954, 'colsample_bytree': 0.5505681952657433}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:22:22,311] Trial 29 finished with value: 0.8580339041980176 and parameters: {'n_estimators': 473, 'max_depth': 17, 'learning_rate': 0.004984577321497195, 'subsample': 0.6142242997530368, 'colsample_bytree': 0.579363706350672}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:22:40,813] Trial 30 finished with value: 0.8617611869143487 and parameters: {'n_estimators': 696, 'max_depth': 20, 'learning_rate': 0.05239872116221006, 'subsample': 0.6909048265283438, 'colsample_bytree': 0.5002411323032582}. Best is trial 10 with value: 0.8630436304641831.
[I 2023-09-22 15:23:15,589] Trial 31 finished with value: 0.862770544775411 and parameters: {'n_estimators': 910, 'max_dept

Best Parameters: {'n_estimators': 988, 'max_depth': 9, 'learning_rate': 0.021746182242880132, 'subsample': 0.7788724729574976, 'colsample_bytree': 0.7822599132555701}
Best F1-Score: 0.8630436304641831


### 4.2.1 Create submission.csv

In [70]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.25, stratify=y_data)

In [71]:
xgb = XGBClassifier(**study.best_params)
xgb.fit(x_train.values, y_train.values, verbose=2)

[17:44:33] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_hidden_layers", "num_units" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.016976047514993316, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_hidden_layers=3,
              num_parallel_tree=1, num_units=240, objective='binary:logistic',
              predictor='auto', ...)

In [72]:
prediction = xgb.predict_proba(x_test)[:,1]
len(prediction)

90954

In [73]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = prediction
result

,id,Machine failure
0,136429,0.093477
1,136430,0.093477
2,136431,0.093477
3,136432,0.093477
4,136433,0.093477
...,...,...
90949,227378,0.093477
90950,227379,0.093477
90951,227380,0.093477
90952,227381,0.093477


In [74]:
result.to_csv('../data/submission.csv', index = False)

## 4.3 TensorFlow

In [37]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.001, restore_best_weights=True)

In [47]:
def objective(trial):
    # 하이퍼파라미터 추천
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 3)
    num_units = trial.suggest_int('num_units', 32, 512, log=True)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    
    f1_scores = []  # 각 폴드의 F1 스코어를 저장할 리스트

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, test_index in skf.split(x_data, y_data):
        x_train, x_valid = x_data.iloc[train_index], x_data.iloc[test_index]
        y_train, y_valid = y_data.iloc[train_index], y_data.iloc[test_index]

        # 모델 구성
        model = Sequential()
        model.add(Dense(num_units, activation='relu', input_shape=(x_data.shape[1],)))
        for i in range(1, num_hidden_layers):
            model.add(Dense(num_units, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))

        # 모델 컴파일
        model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['binary_accuracy'])

        # 모델 훈련 및 평가
        history = model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping], batch_size=len(x_train), verbose=0)

        # 검증 데이터에 대한 예측을 생성
        y_pred = model.predict(x_valid)
        y_pred = (y_pred > 0.5).astype(int)

        # F1 스코어 계산 및 저장
        val_f1_score = f1_score(y_valid, y_pred)
        f1_scores.append(val_f1_score)

    # 각 폴드의 F1 스코어 평균을 반환
    mean_f1_score = sum(f1_scores) / len(f1_scores)
    trial.report(mean_f1_score, step=trial.number)
    
    return mean_f1_score

In [48]:
study = optuna.create_study(direction='maximize')  # 최대화 문제로 설정
study.optimize(objective, n_trials=50)  # n_trials은 시도할 하이퍼파라미터 조합의 수

best_params = study.best_params
best_accuracy = study.best_value

print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_accuracy)

[I 2023-09-22 17:12:17,137] A new study created in memory with name: no-name-33ca5794-0e81-4dbd-9f08-3f145f65e97f
[I 2023-09-22 17:12:37,998] Trial 0 finished with value: 0.8620165761681704 and parameters: {'num_hidden_layers': 3, 'num_units': 77, 'learning_rate': 0.010864338382248874}. Best is trial 0 with value: 0.8620165761681704.
[I 2023-09-22 17:13:02,101] Trial 1 finished with value: 0.0 and parameters: {'num_hidden_layers': 3, 'num_units': 124, 'learning_rate': 0.00012762730383667498}. Best is trial 0 with value: 0.8620165761681704.
[I 2023-09-22 17:13:16,484] Trial 2 finished with value: 0.1977524464686446 and parameters: {'num_hidden_layers': 3, 'num_units': 49, 'learning_rate': 0.0031829713616924934}. Best is trial 0 with value: 0.8620165761681704.
[I 2023-09-22 17:13:32,604] Trial 3 finished with value: 0.2663119664166784 and parameters: {'num_hidden_layers': 3, 'num_units': 35, 'learning_rate': 0.004279649834609575}. Best is trial 0 with value: 0.8620165761681704.
[I 2023-0

[I 2023-09-22 17:28:30,199] Trial 37 finished with value: 0.8619922421376291 and parameters: {'num_hidden_layers': 3, 'num_units': 273, 'learning_rate': 0.004942365866855231}. Best is trial 9 with value: 0.8634400141364752.
[I 2023-09-22 17:29:16,941] Trial 38 finished with value: 0.8611866630656875 and parameters: {'num_hidden_layers': 3, 'num_units': 418, 'learning_rate': 0.0024183175813457286}. Best is trial 9 with value: 0.8634400141364752.
[I 2023-09-22 17:29:39,835] Trial 39 finished with value: 0.1338547767819516 and parameters: {'num_hidden_layers': 1, 'num_units': 343, 'learning_rate': 0.0014405613692050324}. Best is trial 9 with value: 0.8634400141364752.
[I 2023-09-22 17:30:19,239] Trial 40 finished with value: 0.8609670171893675 and parameters: {'num_hidden_layers': 2, 'num_units': 511, 'learning_rate': 0.00322895420503748}. Best is trial 9 with value: 0.8634400141364752.
[I 2023-09-22 17:31:00,233] Trial 41 finished with value: 0.8631283214453592 and parameters: {'num_hidd

Best Hyperparameters: {'num_hidden_layers': 3, 'num_units': 240, 'learning_rate': 0.016976047514993316}
Best Validation Accuracy: 0.8634400141364752


### 4.3.1 Create submission.csv

In [89]:
# EarlyStopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.0001, restore_best_weights=True)

In [90]:
study.best_params

{'num_hidden_layers': 3,
 'num_units': 240,
 'learning_rate': 0.016976047514993316}

In [91]:
# 모델 구조 정의하기 --- (※3)
model = Sequential()
model.add(Dense(240, activation='relu', input_shape=(x_data.shape[1],)))
model.add(Dense(240, activation='relu'))
model.add(Dense(240, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.016976047514993316), metrics=['binary_accuracy'])

In [92]:
model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid), callbacks=[early_stopping], batch_size=64)

Epoch 1/100
800/800 [==============================] - 4s 5ms/step - loss: 0.0330 - binary_accuracy: 0.9943 - val_loss: 0.0233 - val_binary_accuracy: 0.9963
Epoch 2/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0308 - binary_accuracy: 0.9957 - val_loss: 0.0247 - val_binary_accuracy: 0.9952
Epoch 3/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0286 - binary_accuracy: 0.9955 - val_loss: 0.0236 - val_binary_accuracy: 0.9958
Epoch 4/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0290 - binary_accuracy: 0.9952 - val_loss: 0.0252 - val_binary_accuracy: 0.9958
Epoch 5/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0286 - binary_accuracy: 0.9954 - val_loss: 0.0321 - val_binary_accuracy: 0.9953
Epoch 6/100
800/800 [==============================] - 3s 4ms/step - loss: 0.0277 - binary_accuracy: 0.9953 - val_loss: 0.0298 - val_binary_accuracy: 0.9959
Epoch 7/100
800/800 [==============================] - 3s 

In [93]:
predicted_probabilities = model.predict(x_valid)
predicted_classes = (predicted_probabilities > 0.5).astype(int)
f1score = f1_score(y_valid, predicted_classes)
f1score

0.8679245283018867

In [94]:
predicted_probabilities = model.predict(x_test)
len(predicted_probabilities)

90954

In [95]:
result = test_df.copy()
result = result.drop(result.columns.difference(['id']), axis=1)
result['Machine failure'] = predicted_probabilities
result

,id,Machine failure
0,136429,0.002925
1,136430,0.001448
2,136431,0.002319
3,136432,0.002088
4,136433,0.004348
...,...,...
90949,227378,0.003216
90950,227379,0.000508
90951,227380,0.007549
90952,227381,0.000425


In [96]:
result.to_csv('../data/submission.csv', index = False)